In [1]:
# Google Driveと接続を行います。これを行うことで、Driveにあるデータにアクセスできるようになります。
# 下記セルを実行すると、Googleアカウントのログインを求められますのでログインしてください。
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
# 作業フォルダへの移動を行います。
# 人によって作業場所がことなるので、その場合作業場所を変更してください。
#import os 
#os.chdir('/content/drive/MyDrive/100knock-data_analytics/2章') #ここを変更。

# ２章　小売店のデータでデータ加工を行う１０本ノック

本章では、ある小売店の売上履歴と顧客台帳データを用いて、データ分析の素地となる「データの加工」を習得することが目的です。
実際の現場データは手入力のExcel等、決して綺麗なデータではない事が多いため、
データの揺れや整合性の担保など、汚いデータを取り扱うデータ加工を主体に進めて行きます。

### ノック１１：データを読み込んでみよう

In [3]:
import polars as pl
# 列表示を省略しない
pl.Config.set_tbl_cols(-1)
# 行の表示を省略しない
pl.Config.set_tbl_rows(-1)

polars.config.Config

In [4]:
#import pandas as pd
#uriage_data = pd.read_csv("uriage.csv")
uriage_data = pl.read_csv("uriage.csv")
uriage_data.head()

purchase_date,item_name,item_price,customer_name
str,str,i64,str
"""2019-06-13 18:…","""商品A""",100,"""深井菜々美"""
"""2019-07-13 13:…","""商 品 S""",null,"""浅田賢二"""
"""2019-05-11 19:…","""商 品 a""",null,"""南部慶二"""
"""2019-02-12 23:…","""商品Z""",2600,"""麻生莉緒"""
"""2019-04-22 03:…","""商品a""",null,"""平田鉄二"""


In [5]:
#kokyaku_data = pd.read_excel("kokyaku_daicho.xlsx")
kokyaku_data = pl.read_excel("kokyaku_daicho.xlsx")
kokyaku_data.head()

顧客名,かな,地域,メールアドレス,登録日
str,str,str,str,str
"""須賀ひとみ""","""すが ひとみ""","""H市""","""suga_hitomi@ex…","""2018/01/04"""
"""岡田 敏也""","""おかだ としや""","""E市""","""okada_toshiya@…","""42782"""
"""芳賀 希""","""はが のぞみ""","""A市""","""haga_nozomi@ex…","""2018/01/07"""
"""荻野 愛""","""おぎの あい""","""F市""","""ogino_ai@examp…","""42872"""
"""栗田 憲一""","""くりた けんいち""","""E市""","""kurita_kenichi…","""43127"""


### ノック１２：データの揺れを見てみよう

In [6]:
uriage_data["item_name"].head()

item_name
str
"""商品A"""
"""商 品 S"""
"""商 品 a"""
"""商品Z"""
"""商品a"""
"""商品S"""
"""商品 a"""
"""商品V"""
"""商品O"""


In [7]:
uriage_data["item_price"].head()

item_price
i64
100
null
null
2600
null
1900
null
2200
1500


### ノック１３：データに揺れがあるまま集計しよう

In [8]:
#uriage_data["purchase_date"] = pd.to_datetime(uriage_data["purchase_date"])
#uriage_data["purchase_month"] = uriage_data["purchase_date"].dt.strftime("%Y%m")
uriage_data = uriage_data\
    .replace("purchase_date", uriage_data["purchase_date"].str.strptime(dtype = pl.Datetime, format = "%Y-%m-%d %H:%M:%S"))\
        .with_columns([uriage_data["purchase_date"].dt.strftime("%Y%m").alias("purchase_month")])


#res = uriage_data.pivot_table(index="purchase_month", columns="item_name", aggfunc="size", fill_value=0)
import polars.selectors as cs
res = uriage_data.pivot(values = cs.numeric(), index = "purchase_month", columns = "item_name", aggregate_function = "count")
res

C:\Users\treme\AppData\Local\Temp\ipykernel_16780\3282076281.py:4: DeprecationWarning: `replace` is deprecated. DataFrame.replace is deprecated and will be removed in a future version. Please use
    df = df.with_columns(new_column.alias(column_name))
instead.
  .replace("purchase_date", uriage_data["purchase_date"].str.strptime(dtype = pl.Datetime, format = "%Y-%m-%d %H:%M:%S"))\


purchase_month,商品A,商 品 S,商 品 a,商品Z,商品a,商品S,商品 a,商品V,商品O,商 品U,商品L,商 品V,商 品O,商品C,商品I,商品r,商品X,商品 g,商品R,商品P,商品Q,商品y,商品 A,商品N,商品W,商 品E,商品K,商品B,商品F,商 品s,商品W,商 品 n,商 品F,商品D,商品M,商品Y,商品U,商品H,商品T,商品J,商 品O,商品E,商 品Q,商品S,商品M,商 品T,商品G,商 品G,商品P,商品E,商 品N,商 品Y,商品 J,商品 V,商品 K,商 品V,商 品D,商 品A,商品 F,商品 H,商 品K,商 品T,商品 X,商品 Q,商 品X,商 品H,商 品C,商品 B,商品 O,商品 T,商品v,商品p,商品i,商品 w,商 品 s,商 品 q,商品s,商品l,商品t,商品k,商品g,商品o,商品 R,商品 S,商 品M,商品j,商品d,商品 I,商品 E,商品 o,商品c,商品 v,商品e,商品x,商 品I,商品W,商品X,商品 M,商 品P
str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""201906""",23,null,null,null,null,21,null,19,14,1,15,null,1,10,18,null,14,1,16,17,15,null,1,16,16,null,15,12,18,null,null,null,null,19,17,13,17,12,12,22,null,13,null,null,null,null,13,1,null,null,null,null,null,1,null,null,null,null,null,1,null,null,null,null,null,null,1,null,null,null,null,1,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,null,1,null,null,null
"""201907""",20,1,null,null,null,22,null,26,19,null,17,null,1,17,18,2,18,null,17,26,14,null,null,15,16,1,12,20,17,null,null,null,null,17,11,12,13,18,20,23,1,10,null,null,null,null,19,null,null,null,null,null,null,1,null,1,null,null,null,null,null,null,null,null,null,1,null,null,null,1,null,null,1,null,null,1,null,null,null,null,null,1,null,null,null,null,null,null,1,null,null,null,null,null,null,null,null,null,null
"""201905""",20,null,2,null,null,16,1,8,18,null,12,null,null,15,16,null,16,null,22,20,12,1,null,19,20,null,16,14,17,null,null,null,null,14,17,19,16,15,15,10,null,19,null,1,null,null,23,null,null,null,null,null,1,null,1,1,null,1,1,null,1,null,null,1,null,null,null,null,null,1,null,null,null,1,1,null,null,1,null,null,null,null,null,null,1,null,null,null,null,null,1,null,null,null,null,null,null,null,null
"""201902""",19,null,null,1,null,21,null,21,18,null,11,null,null,26,11,null,14,1,21,19,22,null,null,19,24,null,16,13,14,null,null,null,null,20,15,11,19,17,22,13,null,16,null,null,null,null,13,null,null,null,1,null,null,null,null,null,1,null,null,null,null,null,1,null,null,null,null,1,null,null,1,null,null,null,null,null,1,null,1,null,null,null,1,null,null,1,null,null,null,1,null,null,null,null,1,null,1,null,null
"""201904""",16,null,null,null,1,15,null,14,14,null,15,1,1,24,18,null,16,null,20,15,19,null,null,20,13,null,14,19,15,null,1,null,null,20,11,17,11,11,15,13,null,17,1,null,null,1,13,null,null,null,null,3,null,null,null,null,null,null,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,null,null,null,1,null,null,null,null,null,null,null,null,null,null,null,1,null,null,null,null,null,null
"""201903""",17,null,null,null,null,20,null,17,11,null,20,null,null,20,12,null,21,null,16,20,23,null,null,13,16,null,14,21,26,null,null,null,1,16,21,16,23,18,12,15,null,8,null,null,1,null,14,null,1,1,null,null,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,null,null,null,null,1,null,null,null,null,null,null,null
"""201901""",18,null,null,null,null,18,null,21,21,null,18,null,1,18,17,null,12,null,21,15,17,null,null,15,13,null,19,13,15,1,null,1,null,17,18,10,7,15,16,17,null,18,null,null,null,null,11,null,null,null,null,null,null,1,null,null,null,null,null,null,null,1,null,null,2,1,null,null,1,null,null,null,null,null,null,null,null,1,null,1,null,1,null,1,null,null,null,1,null,null,1,null,null,null,null,null,null,1,1


In [9]:
#res = uriage_data.pivot_table(index="purchase_month", columns="item_name", values="item_price", aggfunc="sum", fill_value=0)
res = uriage_data.pivot(values = "item_price", index = "purchase_month", columns = "item_name", aggregate_function = "sum")
res

purchase_month,商品A,商 品 S,商 品 a,商品Z,商品a,商品S,商品 a,商品V,商品O,商 品U,商品L,商 品V,商 品O,商品C,商品I,商品r,商品X,商品 g,商品R,商品P,商品Q,商品y,商品 A,商品N,商品W,商 品E,商品K,商品B,商品F,商 品s,商品W,商 品 n,商 品F,商品D,商品M,商品Y,商品U,商品H,商品T,商品J,商 品O,商品E,商 品Q,商品S,商品M,商 品T,商品G,商 品G,商品P,商品E,商 品N,商 品Y,商品 J,商品 V,商品 K,商 品V,商 品D,商 品A,商品 F,商品 H,商 品K,商 品T,商品 X,商品 Q,商 品X,商 品H,商 品C,商品 B,商品 O,商品 T,商品v,商品p,商品i,商品 w,商 品 s,商 品 q,商品s,商品l,商品t,商品k,商品g,商品o,商品 R,商品 S,商 品M,商品j,商品d,商品 I,商品 E,商品 o,商品c,商品 v,商品e,商品x,商 品I,商品W,商品X,商品 M,商 品P
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""201906""",2000,null,null,null,null,34200,null,33000,19500,2100,14400,null,1500,2400,12600,null,24000,700,27000,24000,23800,null,100,15400,25300,null,15400,2400,9600,null,null,null,null,7200,19500,30000,29400,8800,22000,20000,null,6500,null,null,null,null,7700,700,null,null,null,null,null,0,null,null,null,null,null,800,null,null,null,null,null,null,300,null,null,null,null,1600,900,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2400,null,2300,null,null,null
"""201907""",1600,0,null,null,null,38000,null,52800,25500,null,20400,null,1500,4500,13500,1800,38400,null,25200,38400,18700,null,null,18200,32200,500,9900,3600,9600,null,null,null,null,6000,13000,27500,23100,12800,34000,17000,0,4500,null,null,null,null,11900,null,null,null,null,null,null,2200,null,2200,null,null,null,null,null,null,null,null,null,800,null,null,null,2000,null,null,900,null,null,1700,null,null,null,null,null,1500,null,null,null,null,null,null,500,null,null,null,null,null,null,null,null,null,null
"""201905""",1900,null,100,null,null,20900,0,8800,24000,null,12000,null,null,3900,13500,null,31200,null,32400,28800,15300,2500,null,23800,36800,null,15400,2600,9000,null,null,null,null,5200,20800,45000,33600,12000,26000,8000,null,9000,null,1900,null,null,14000,null,null,null,null,null,1000,null,1100,2200,null,100,600,null,1100,null,null,1700,null,null,null,null,null,2000,null,null,null,2300,1900,null,null,1200,null,null,null,null,null,null,1300,null,null,null,null,null,300,null,null,null,null,null,null,null,null
"""201902""",1700,null,null,2600,null,32300,null,39600,25500,null,13200,null,null,6000,8100,null,31200,700,32400,30400,30600,null,null,23800,43700,null,14300,2400,6600,null,null,null,null,7600,16900,20000,37800,12800,40000,11000,null,5500,null,null,null,null,7000,null,null,null,1400,null,null,null,null,null,400,null,null,null,null,null,2400,null,null,null,null,0,null,null,2200,null,null,null,null,null,1900,null,2000,null,null,null,1800,null,null,1000,null,null,null,1500,null,null,null,null,900,null,2400,null,null
"""201904""",1400,null,null,null,0,28500,null,19800,18000,null,15600,2200,1500,6000,14400,null,38400,null,30600,22400,28900,null,null,22400,20700,null,12100,2200,7800,null,2300,null,null,7200,14300,32500,16800,7200,28000,13000,null,8000,1700,null,null,2000,9100,null,null,null,null,7500,null,null,null,null,null,null,1200,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1900,null,null,null,700,null,null,null,null,null,null,null,null,null,null,null,500,null,null,null,null,null,null
"""201903""",1300,null,null,null,null,26600,null,35200,15000,null,21600,null,null,5100,10800,null,38400,null,27000,32000,34000,null,null,16800,34500,null,14300,4200,11400,null,null,null,600,6000,26000,35000,46200,14400,22000,13000,null,4000,null,null,1300,null,7700,null,1600,500,null,null,1000,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,400,null,null,null,null,0,null,null,null,null,null,null,null
"""201901""",1500,null,nu

### ノック１４：商品名の揺れを補正しよう

In [10]:
#print(len(pd.unique(uriage_data["item_name"])))
print( len( uriage_data["item_name"].unique() ) )

99


In [11]:
uriage_data["item_name"].unique().sort().head()
# 商品名の揺れ
# 半角スペース "  商品W"
# 半角スペース 2個連続 "商  品O"
# 小文字のアルファベット " 商 品 n"

item_name
str
""" 商品W"""
""" 商 品 n"""
""" 商品E"""
""" 商品M"""
""" 商品P"""
""" 商品S"""
""" 商品W"""
""" 商品X"""
"""商 品O"""


In [12]:
#uriage_data["item_name"] = uriage_data["item_name"].str.upper()
#uriage_data["item_name"] = uriage_data["item_name"].str.replace("　", "")
#uriage_data["item_name"] = uriage_data["item_name"].str.replace(" ", "")
uriage_data = uriage_data\
    .replace("item_name",
             uriage_data["item_name"]\
                .str.to_uppercase()\
                    .str.replace_all("　", "")\
                        .str.replace_all(" ", ""))

#uriage_data.sort_values(by=["item_name"], ascending=True)
uriage_data.sort(by = ["item_name"]).head(10)

C:\Users\treme\AppData\Local\Temp\ipykernel_16780\868117454.py:5: DeprecationWarning: `replace` is deprecated. DataFrame.replace is deprecated and will be removed in a future version. Please use
    df = df.with_columns(new_column.alias(column_name))
instead.
  .replace("item_name",


purchase_date,item_name,item_price,customer_name,purchase_month
datetime[μs],str,i64,str,str
2019-06-13 18:02:34,"""商品A""",100,"""深井菜々美""","""201906"""
2019-05-11 19:42:07,"""商品A""",null,"""南部慶二""","""201905"""
2019-04-22 03:09:35,"""商品A""",null,"""平田鉄二""","""201904"""
2019-05-18 19:16:53,"""商品A""",null,"""深井照生""","""201905"""
2019-01-28 10:47:03,"""商品A""",100,"""大地礼子""","""201901"""
2019-06-11 12:57:24,"""商品A""",null,"""黄川田博之""","""201906"""
2019-06-29 10:01:52,"""商品A""",100,"""石田佑""","""201906"""
2019-01-12 03:26:36,"""商品A""",100,"""小平陽子""","""201901"""
2019-02-11 17:00:37,"""商品A""",100,"""原口俊二""","""201902"""


In [18]:
#print(pd.unique(uriage_data["item_name"]))
#print(len(pd.unique(uriage_data["item_name"])))
#print( uriage_data["item_name"].unique() )
uriage_data["item_name"].unique()
#print( len( uriage_data["item_name"].unique() ) )

item_name
str
"""商品R"""
"""商品Y"""
"""商品Q"""
"""商品J"""
"""商品C"""
"""商品K"""
"""商品G"""
"""商品T"""
"""商品V"""


In [14]:
len( uriage_data["item_name"].unique() )

26

### ノック１５：金額欠損値の補完をしよう

In [15]:
#uriage_data.isnull().any(axis=0)
uriage_data.select(pl.all().null_count())

purchase_date,item_name,item_price,customer_name,purchase_month
u32,u32,u32,u32,u32
0,0,387,0,0


In [16]:
flg_is_null = uriage_data["item_price"].isnull()
for trg in list(uriage_data.loc[flg_is_null, "item_name"].unique()):
    price = uriage_data.loc[(~flg_is_null) & (uriage_data["item_name"] == trg), "item_price"].max()
    uriage_data.loc[(flg_is_null) & (uriage_data["item_name"]==trg),"item_price"] = price
uriage_data.head()

# 製品価格の欠損値を埋めるために、{製品名: 製品価格}の辞書型を作成する
uriage_drop_null = uriage_data[["item_name", "item_price"]]\
    .sort(by = ["item_price"], descending = True)\
        .drop_nulls(subset = ["item_price"])\
            .unique(subset = ["item_name"])

uriage_dict = {}
for row in range(len(uriage_drop_null)):
    uriage_dict[uriage_drop_null[row, 0]] = uriage_drop_null[row, 1]
uriage_dict

del uriage_drop_null




AttributeError: 'Series' object has no attribute 'isnull'

In [25]:
uriage_drop_null = uriage_data[["item_name", "item_price"]]\
    .sort(by = ["item_price"], descending = True)\
        .drop_nulls(subset = ["item_price"])\
            .unique(subset = ["item_name"])
uriage_dict = {}
for row in range(len(uriage_drop_null)):
    uriage_dict[uriage_drop_null[row, 0]] = uriage_drop_null[row, 1]
uriage_dict

{'商品B': 200,
 '商品E': 500,
 '商品Q': 1700,
 '商品F': 600,
 '商品O': 1500,
 '商品K': 1100,
 '商品W': 2300,
 '商品C': 300,
 '商品G': 700,
 '商品A': 100,
 '商品R': 1800,
 '商品L': 1200,
 '商品X': 2400,
 '商品H': 800,
 '商品J': 1000,
 '商品D': 400,
 '商品Y': 2500,
 '商品N': 1400,
 '商品V': 2200,
 '商品Z': 2600,
 '商品T': 2000,
 '商品P': 1600,
 '商品U': 2100,
 '商品S': 1900,
 '商品I': 900,
 '商品M': 1300}

In [ ]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_month    False
dtype: bool

In [ ]:
for trg in list(uriage_data["item_name"].sort_values().unique()):
    print(trg + "の最大額：" + str(uriage_data.loc[uriage_data["item_name"]==trg]["item_price"].max()) + "の最小額：" + str(uriage_data.loc[uriage_data["item_name"]==trg]["item_price"].min(skipna=False)))

商品Aの最大額：100.0の最小額：100.0
商品Bの最大額：200.0の最小額：200.0
商品Cの最大額：300.0の最小額：300.0
商品Dの最大額：400.0の最小額：400.0
商品Eの最大額：500.0の最小額：500.0
商品Fの最大額：600.0の最小額：600.0
商品Gの最大額：700.0の最小額：700.0
商品Hの最大額：800.0の最小額：800.0
商品Iの最大額：900.0の最小額：900.0
商品Jの最大額：1000.0の最小額：1000.0
商品Kの最大額：1100.0の最小額：1100.0
商品Lの最大額：1200.0の最小額：1200.0
商品Mの最大額：1300.0の最小額：1300.0
商品Nの最大額：1400.0の最小額：1400.0
商品Oの最大額：1500.0の最小額：1500.0
商品Pの最大額：1600.0の最小額：1600.0
商品Qの最大額：1700.0の最小額：1700.0
商品Rの最大額：1800.0の最小額：1800.0
商品Sの最大額：1900.0の最小額：1900.0
商品Tの最大額：2000.0の最小額：2000.0
商品Uの最大額：2100.0の最小額：2100.0
商品Vの最大額：2200.0の最小額：2200.0
商品Wの最大額：2300.0の最小額：2300.0
商品Xの最大額：2400.0の最小額：2400.0
商品Yの最大額：2500.0の最小額：2500.0
商品Zの最大額：2600.0の最小額：2600.0


### ノック１６：顧客名の揺れを補正しよう

In [ ]:
kokyaku_data["顧客名"].head()

0     須賀ひとみ
1    岡田　 敏也
2      芳賀 希
3     荻野  愛
4     栗田 憲一
Name: 顧客名, dtype: object

In [ ]:
uriage_data["customer_name"].head()

0    深井菜々美
1     浅田賢二
2     南部慶二
3     麻生莉緒
4     平田鉄二
Name: customer_name, dtype: object

In [ ]:
kokyaku_data["顧客名"] = kokyaku_data["顧客名"].str.replace("　", "")
kokyaku_data["顧客名"] = kokyaku_data["顧客名"].str.replace(" ", "")
kokyaku_data["顧客名"].head()

0    須賀ひとみ
1     岡田敏也
2      芳賀希
3      荻野愛
4     栗田憲一
Name: 顧客名, dtype: object

### ノック１７：日付の揺れを補正しよう

In [ ]:
flg_is_serial = kokyaku_data["登録日"].astype("str").str.isdigit()
flg_is_serial.sum()

22

In [ ]:
fromSerial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, "登録日"].astype("float") - 2, unit="D") + pd.to_datetime('1900/1/1')
fromSerial

1     2017-02-16
3     2017-05-17
4     2018-01-27
21    2017-07-04
27    2017-06-15
47    2017-01-06
49    2017-07-13
53    2017-04-08
76    2018-03-29
80    2018-01-10
99    2017-05-30
114   2018-06-03
118   2018-01-29
122   2018-04-16
139   2017-05-25
143   2017-03-24
155   2017-01-19
172   2018-03-22
179   2017-01-08
183   2017-07-24
186   2018-07-13
192   2018-06-08
Name: 登録日, dtype: datetime64[ns]

In [ ]:
fromString = pd.to_datetime(kokyaku_data.loc[~flg_is_serial, "登録日"])
fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 登録日, Length: 178, dtype: datetime64[ns]

In [ ]:
kokyaku_data["登録日"] = pd.concat([fromSerial, fromString])
kokyaku_data

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-16
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-17
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-27
...,...,...,...,...,...
195,川上りえ,かわかみ りえ,G市,kawakami_rie@example.com,2017-06-20
196,小松季衣,こまつ としえ,E市,komatsu_toshie@example.com,2018-06-20
197,白鳥りえ,しらとり りえ,F市,shiratori_rie@example.com,2017-04-29
198,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019-04-19


In [ ]:
kokyaku_data["登録年月"] = kokyaku_data["登録日"].dt.strftime("%Y%m")
rslt = kokyaku_data.groupby("登録年月").count()["顧客名"]
print(rslt)
print(len(kokyaku_data))

登録年月
201701    15
201702    11
201703    14
201704    15
201705    14
201706    13
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 顧客名, dtype: int64
200


In [ ]:
flg_is_serial = kokyaku_data["登録日"].astype("str").str.isdigit()
flg_is_serial.sum()

0

### ノック１８：顧客名をキーに２つのデータを結合(ジョイン)しよう

In [ ]:
join_data = pd.merge(uriage_data, kokyaku_data, left_on="customer_name", right_on="顧客名", how="left")
join_data = join_data.drop("customer_name", axis=1)
join_data

,purchase_date,item_name,item_price,purchase_month,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-06-13 18:02:34,商品A,100.0,201906,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,201701
1,2019-07-13 13:05:29,商品S,1900.0,201907,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07,201804
2,2019-05-11 19:42:07,商品A,100.0,201905,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19,201806
3,2019-02-12 23:40:45,商品Z,2600.0,201902,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22,201807
4,2019-04-22 03:09:35,商品A,100.0,201904,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07,201706
...,...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:39,商品Y,2500.0,201902,福島友也,ふくしま ともや,B市,fukushima_tomoya@example.com,2017-07-01,201707
2995,2019-06-22 04:03:43,商品M,1300.0,201906,大倉晃司,おおくら こうじ,E市,ookura_kouji@example.com,2018-03-31,201803
2996,2019-03-29 11:14:05,商品Q,1700.0,201903,尾形小雁,おがた こがん,B市,ogata_kogan@example.com,2017-03-15,201703
2997,2019-07-14 12:56:49,商品H,800.0,201907,芦田博之,あしだ ひろゆき,E市,ashida_hiroyuki@example.com,2018-07-13,201807


### ノック１９：クレンジングしたデータをダンプしよう

In [ ]:
dump_data = join_data[["purchase_date", "purchase_month", "item_name", "item_price", "顧客名", "かな", "地域", "メールアドレス", "登録日"]]
dump_data

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,201906,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26
1,2019-07-13 13:05:29,201907,商品S,1900.0,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07
2,2019-05-11 19:42:07,201905,商品A,100.0,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19
3,2019-02-12 23:40:45,201902,商品Z,2600.0,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22
4,2019-04-22 03:09:35,201904,商品A,100.0,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07
...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:39,201902,商品Y,2500.0,福島友也,ふくしま ともや,B市,fukushima_tomoya@example.com,2017-07-01
2995,2019-06-22 04:03:43,201906,商品M,1300.0,大倉晃司,おおくら こうじ,E市,ookura_kouji@example.com,2018-03-31
2996,2019-03-29 11:14:05,201903,商品Q,1700.0,尾形小雁,おがた こがん,B市,ogata_kogan@example.com,2017-03-15
2997,2019-07-14 12:56:49,201907,商品H,800.0,芦田博之,あしだ ひろゆき,E市,ashida_hiroyuki@example.com,2018-07-13


In [ ]:
dump_data.to_csv("dump_data.csv", index=False)

### ノック２０：データを集計しよう

In [ ]:
import_data = pd.read_csv("dump_data.csv")
import_data

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,201906,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26
1,2019-07-13 13:05:29,201907,商品S,1900.0,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07
2,2019-05-11 19:42:07,201905,商品A,100.0,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19
3,2019-02-12 23:40:45,201902,商品Z,2600.0,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22
4,2019-04-22 03:09:35,201904,商品A,100.0,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07
...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:39,201902,商品Y,2500.0,福島友也,ふくしま ともや,B市,fukushima_tomoya@example.com,2017-07-01
2995,2019-06-22 04:03:43,201906,商品M,1300.0,大倉晃司,おおくら こうじ,E市,ookura_kouji@example.com,2018-03-31
2996,2019-03-29 11:14:05,201903,商品Q,1700.0,尾形小雁,おがた こがん,B市,ogata_kogan@example.com,2017-03-15
2997,2019-07-14 12:56:49,201907,商品H,800.0,芦田博之,あしだ ひろゆき,E市,ashida_hiroyuki@example.com,2018-07-13


In [ ]:
byItem = import_data.pivot_table(index="purchase_month", columns="item_name", aggfunc="size", fill_value=0)
byItem

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
201902,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
201903,17,21,20,17,9,27,14,18,12,16,...,23,16,20,12,23,18,16,21,16,0
201904,17,19,24,20,18,17,14,11,18,13,...,20,20,16,16,11,15,14,16,20,0
201905,24,14,16,14,19,18,23,15,16,11,...,13,22,18,16,16,9,21,16,20,0
201906,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
201907,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


In [ ]:
byPrice = import_data.pivot_table(index="purchase_month", columns="item_name", values="item_price", aggfunc="sum", fill_value=0)
byPrice

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1800.0,2600.0,5700.0,6800.0,9000.0,9000.0,7700.0,12800.0,16200.0,17000.0,...,28900.0,37800.0,38000.0,34000.0,14700.0,48400.0,29900.0,33600.0,25000.0,0.0
201902,1900.0,2800.0,7800.0,8400.0,8000.0,8400.0,9800.0,13600.0,10800.0,14000.0,...,37400.0,39600.0,41800.0,46000.0,39900.0,48400.0,55200.0,38400.0,27500.0,2600.0
201903,1700.0,4200.0,6000.0,6800.0,4500.0,16200.0,9800.0,14400.0,10800.0,16000.0,...,39100.0,28800.0,38000.0,24000.0,48300.0,39600.0,36800.0,50400.0,40000.0,0.0
201904,1700.0,3800.0,7200.0,8000.0,9000.0,10200.0,9800.0,8800.0,16200.0,13000.0,...,34000.0,36000.0,30400.0,32000.0,23100.0,33000.0,32200.0,38400.0,50000.0,0.0
201905,2400.0,2800.0,4800.0,5600.0,9500.0,10800.0,16100.0,12000.0,14400.0,11000.0,...,22100.0,39600.0,34200.0,32000.0,33600.0,19800.0,48300.0,38400.0,50000.0,0.0
201906,2400.0,2400.0,3300.0,7600.0,6500.0,10800.0,10500.0,10400.0,17100.0,22000.0,...,25500.0,28800.0,39900.0,24000.0,37800.0,44000.0,39100.0,36000.0,32500.0,0.0
201907,2000.0,4000.0,5100.0,6800.0,6000.0,10200.0,13300.0,15200.0,17100.0,23000.0,...,25500.0,34200.0,43700.0,42000.0,27300.0,61600.0,36800.0,43200.0,30000.0,0.0


In [ ]:
byCustomer = import_data.pivot_table(index="purchase_month", columns="顧客名", aggfunc="size", fill_value=0)
byCustomer

顧客名,さだ千佳子,中仁晶,中田美智子,丸山光臣,久保田倫子,亀井一徳,五十嵐春樹,井上桃子,井口寛治,井川真悠子,...,香椎優一,高原充則,高梨結衣,高沢美咲,高田さんま,鳥居広司,鶴岡薫,麻生莉緒,黄川田博之,黒谷長利
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,3,1,4,2,2,0,5,3,3,1,...,0,1,1,1,5,2,0,2,2,5
201902,9,1,2,2,1,4,2,1,0,4,...,4,0,3,2,0,1,2,4,0,1
201903,1,2,1,6,1,4,3,3,2,2,...,3,1,6,2,4,2,4,2,2,1
201904,0,3,1,2,0,2,2,0,3,2,...,2,4,2,3,4,3,2,1,2,0
201905,3,2,5,2,4,1,2,1,3,3,...,1,1,1,0,2,2,3,4,4,1
201906,1,3,0,4,1,1,1,2,2,3,...,7,3,0,2,1,0,2,1,2,4
201907,3,0,3,2,5,3,5,2,5,5,...,2,4,4,2,0,2,4,3,4,1


In [ ]:
byRegion = import_data.pivot_table(index="purchase_month", columns="地域", aggfunc="size", fill_value=0)
byRegion

地域,A市,B市,C市,D市,E市,F市,G市,H市
purchase_month,,,,,,,,
201901,59,55,72,34,49,57,49,42
201902,71,46,65,48,61,52,43,63
201903,64,52,57,43,52,59,51,59
201904,64,48,54,45,48,58,40,52
201905,57,52,68,48,59,65,35,43
201906,53,47,61,30,51,51,58,58
201907,76,53,61,42,54,64,47,54


In [ ]:
away_data = pd.merge(uriage_data, kokyaku_data, left_on="customer_name", right_on="顧客名", how="right")
away_data[away_data["purchase_date"].isnull()][["顧客名", "メールアドレス", "登録日"]]

,顧客名,メールアドレス,登録日
2999,福井美希,fukui_miki1@example.com,2019-04-23
